위 글은 모두의 연구소 AIFFEL Going Deeper node를 참고하여 만들었음을 밝힙니다.

**XAI(Explainable Artificial Intelligence)** 
===

목차

1. Explainable AI
2. CAM: Class Activation Map
3. Grad-CAM
4. ACoL: Adversarial Complementary Learning

## 1. Explainable AI
---

**XAI(explainable AI)** 

모델이 잘못된 답변을 준다면 어떻게 개선할 수 있을지, 잘 동작한다면 왜 이런 선택을 했는지 알고자 하는 것 




#### 이미지 분류 문제(Image Classification)
---

![image](https://user-images.githubusercontent.com/70703320/115188720-4b57c200-a120-11eb-80e3-b7473e784ddd.png)

https://3months.tistory.com/465

분류모델에 이미지가 입력으로 들어온다고 해보자. 먼저 일반적인 이미지 분류 모델은 이미지의 local feature를 추출하기 위해 CNN으로 구성된 특성 추출용 백본 네트워크(backbone network)가 앞에 있다. 백본 네트워크에서 추출된 특성 맵(feature map)을 fully connected layer에 통과시켜 얻어진 logit을 소프트맥스(softmax) 활성화 함수에 통과시키면, 입력 이미지가 각 클래스에 속할 확률을 얻을 수 있다.

logit이 아래와 같이 sigmoid(logistic function)의 역함수이며, softmax는 sigmoid를 K개의 클래스로 일반화한 것이다.
 
레이어마다 feature map을 시각화해서, activation을 확인하여 이미지 분류 근거를 찾을 수 있다.

## 2. CAM: Class Activation Map

**CAM(Class Activation Map)** 

"모델이 어떤 곳을 보고 각 클래스임을 짐작하고 있는지" 확인할 수 있는 지도

MIT 연구진의 [Learning Deep Features for Discriminative Localization](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf) 논문


GAP(global average pooling) 기법

어떤 클래스가 어느 영역에 의해서 활성화되었는지 알 수 있다.

### 1) GAP(Global Average Pooling)
---

![image](https://user-images.githubusercontent.com/70703320/115192636-08005200-a126-11eb-85f0-d5a3b61c27db.png)
\[출처 : http://taewan.kim/post/cnn/\]


**Average pooling과 Max pooling**

Average pooling과 Max pooling은 각각 커널과 겹치는 영역에 대해 Average 값을 취한 것과 Max 값을 취한 것


![image](https://user-images.githubusercontent.com/70703320/115192791-3c740e00-a126-11eb-8dc9-1a704a4675a5.png)

**이전**

1. 일반적으로 이미지 분류를 위해서 여러 CNN 레이어를 거쳐 우선 특성을 추출
2. 그 특성 맵을 flattening 한 후 fully connected layer에 입력해 줌
3. 각 클래스에 따른 logit을 구하고 최종 활성화 함수를 거침

**GAP**

1. 매 채널별로 average pooling을 채널의 값 전체에 global하게 적용
2. 크기가 6x6이고 채널이 3개인 특성맵에 대해서 GAP을 수행
3. 각 채널이 딱 한 개의 숫자로 요약되어, 1x1 크기에 채널이 3개인 벡터를 얻게 됨
4. 이때 결과 벡터의 각 차원의 값은 6x6 크기의 특성 맵을 채널 별로 평균을 낸 값이 됨 

**GAP의 장점**

마지막 CNN 레이어의 채널 수는 데이터의 클래스 수에 맞춰 각 클래스에 따른 확률을 얻을 수 있도록 했다. 

1. **각 채널이 클래스별 신뢰도를 나타내게 되어 해석이 쉽다** 
2. **fully connected layer과 달리 최적화할 파라미터가 존재하지 않으므로 과적합(overfitting)을 방지할 수 있다**

![image](https://user-images.githubusercontent.com/70703320/115487736-4156d000-a294-11eb-8c18-b00420e54dc9.png)


https://arxiv.org/abs/1312.4400


### 2) CAM
---

**CAM(Class Activation Map)** 

클래스가 활성화되는 지도

일반적인 CNN은 커널 윈도우에 따라서 특성을 추출하므로 CNN 레이어를 거친 특성 맵에도 입력값의 위치정보가 유지

특성 맵의 정보를 이미지 검출(detection)나 세그멘테이션(Segmentation) 등의 문제를 푸는 데 이용


![image](https://user-images.githubusercontent.com/70703320/115192878-5f9ebd80-a126-11eb-9c33-e74d54f09484.png)
http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf


**CAM을 얻는 과정**

1. CNN 레이어를 거쳐 뽑아낸 특성 맵에 대해서 GAP을 적용
2. 이에 대해서 소프트맥스 레이어(소프트맥스 활성화 함수를 가지고 bias가 없는 fully connected layer)를 적용
3. CNN을 거친 특성맵에서 각 클래스에 대한 정보는 결과값의 여러 채널에 걸쳐 나타나게 됨
4. GAP을 통해 각 채널 별 정보를 요약하면 소프트맥스 레이어는 이 정보를 보고 각 클래스에 대한 개별 채널의 중요도를 결정
5. 클래스 별로 소프트맥스 레이어를 통해 각 채널의 가중합을 구하면 각 클래스가 활성화 맵의 어떤 부분을 주로 활성화 시키는지 확인할 수 있음
6. 특성 맵은 CNN의 출력값 크기와 같게 되는데 보간(interpolation)을 통해 적절히 원본 이미지 크기로 확대해 주면 위와 같은 CAM을 얻을 수 있음


k = 1, 2, ..., n 인 k번째 채널에 대해서,$w_{k}^{c}$는 각 클래스 (c)노드와 k번째 채널 사이의 가중치 값이다. $f_k(x,y)$는 k번째 채널의 $x, y$ 요소의 활성화 값이다. 위 그림에서는 파란색 네모 박스로 시각화되어 있다.


이 두 가지를 곱하고 모든 채널과 x,y 축에 대해 더해줌으로써 클래스별 점수를 구할 수 있다. 이렇게 구한 것이 위 그림 우하단의 최종 Class Activation Map으로 시각화되어 있다.

![image](https://user-images.githubusercontent.com/70703320/115195783-25cfb600-a12a-11eb-8edf-fa2b7064dbb0.png)

http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf

CAM을 얻기 위해서는 위에서 점수를 얻기 위해서 모든 위치의 활성화 정도를 더해준 것과 달리 각 $x, y$ 위치에서 $k$개의 채널만 더해주어 위치정보가 남도록 한다. 이를 통해서 얻어진 CAM은 각 위치에서 채널 별 활성화 정도의 가중합인 $M_c(x, y)$가 된다.$M_c(x,y)$은 모델이 클래스 c에 대해 각 위치를 얼마나 보고 있는지 나타내는 것이다.

![image](https://user-images.githubusercontent.com/70703320/115195820-2e27f100-a12a-11eb-8f64-399618b0a34c.png)

http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf

## 3. Grad-CAM

**Grad-CAM의 특징**

1. CNN 기반의 네트워크가 **모델 구조를 변경할 필요가 없어진다.** 
2. 분류 문제 외의 **다른 태스크들에 유연하게 대처**할 수 있다.

![image](https://user-images.githubusercontent.com/70703320/115196176-98409600-a12a-11eb-8660-d592ce85e955.png)
https://arxiv.org/pdf/1610.02391.pdf

개와 고양이가 함께 있는 사진에서 개와 고양이 각각에 대해  Guided Backprop, Grad-CAM, Occlussion map이 어떻게 나타나는지 비교한 것

Grad-CAM에서는 높은 분별력과 큰 dimension을 갖는 CAM을 만드는 것을 중요하게 보았다.

![image](https://user-images.githubusercontent.com/70703320/115196245-ad1d2980-a12a-11eb-8701-51ce5f09b62e.png)
https://arxiv.org/pdf/1610.02391.pdf

**Grad-CAM의 전체적인 구조**

- 이미지를 입력으로 CNN을 거쳐 특성 맵을 추출
- 이어 태스크에 따라서 다양한 레이어들이 사용됨

**Image Classificatio** ::

**Image Captioning** :: 이미지에 대한 설명을 만들어내는 태스크

**Visual question answering** :: VQA로 어떤 질문과 이미지가 주어졌을 때 이에 대한 답변을 내는 태스크

Grad-CAM :: 복잡한 태스크에 사용되는 모델에서도 사용될 수 있다는 점


### 그래디언트를 통한 Weight Score 계산
---
**CAM**
소프트맥스를 가진 fully connected layer의 가중치를 통해서 어떤 클래스에 대한 각 채널의 중요도 또는 가중치를 얻어낸다.

**Grad-CAM**
원하는 클래스에 대해서 관찰하는 레이어로 들어오는 그래디언트를 구하여, 해당 클래스를 활성화하는 데 레이어의 특성 맵에서 어떤 채널이 중요하게 작용하는지 알 수 있다.


클래스에 대해서 backpropagation을 통해 얻은 $k$번째 채널의 그래디언트를 통해 가중치 점수(Weight score)를 구하는 식

![image](https://user-images.githubusercontent.com/70703320/115734224-7e21e480-a3c4-11eb-8d48-c89b9066ffd4.png)

- $y$는 모델의 출력값이고 
- $A$는 활성화 맵을 의미 
- $i,j$는 각각 x축, y축으로 생각
- $Z$는 전체 map의 크기

$i = 1, 2, ..., u$ 이고,  $j = 1, 2, ..., v$라면 $Z = u*v$ 가 되어 활성화 맵 전체에 대한 global average를 구하기 위한 분모가 된다.

![image](https://user-images.githubusercontent.com/70703320/115197275-c4a8e200-a12b-11eb-9c1f-3c201c1f5304.png)
https://arxiv.org/pdf/1610.02391.pdf

$k$개의 채널을 가진 활성화 맵에서 각 채널이 어떤 클래스를 활성화하는 데 얼마나 중요하게 작용하는지 가중치 점수를 구할 수 있다. 

이는 **이 가중치를 구하기 위해 CAM처럼 별도의 weight 파라미터를 도입할 필요가 없다** 뜻이다.

이 정보를 합쳐 활성화 맵에서 어떤 클래스의 위치에 따른 활성화를 보려면 

- $k$ 번째 활성화 맵과 가중치를 곱해주어 합함
- ReLU 활성화 함수를 통해서 클래스에 따른 Grad-CAM을 얻음
- 이 수식을 시각화한 것이 맨 위에서 소개한 고양이와 개에 대한 Grad-CAM의 (c), (i)번째 예시이다. 

![image](https://user-images.githubusercontent.com/70703320/115197340-d4c0c180-a12b-11eb-976e-7037410ef881.png)
https://arxiv.org/pdf/1610.02391.pdf

## 4. ACoL: Adversarial Complementary Learning


**CAM**
클래스와 이미지만을 데이터로 학습을 하는데 덤으로 위치정보까지 얻을 수 있다. 

**약지도학습(weakly supervised learning)**
직접적으로 정답 위치정보를 주지 않아도 간접적인 정보를 활용하여 학습을 하고 원하는 정보를 얻어낼 수 있도록 모델을 학습하는 방식

`CAM`, `Grad-CAM`,`ACoL`은 약지도학습 기법을 활용한 물체 검출(object detection)을 수행할 수 있다.

### 1) 약지도학습(weakly supervised learning)
---

- **incomplete supervision :** 학습 데이터 중 일부에만 라벨이 달려 있는 경우 (예: 개와 고양이 분류 학습시 10000개의 이미지 중 1000개만 라벨이 있는 경우) 일반적으로 말하는 준지도학습과 같은 경우이다.

- **inexact supervision :** 학습데이터의 라벨이 충분히 정확하게 달려있지 않은 경우. (예: 개나 고양이를 Object Detection 또는 Semantic Segmentation해야 하는데, 이미지 내에 정확한 bounding box는 주어져 있지 않고 이미지가 개인지 고양인지 정보만 라벨로 달려있는 경우)

- **inaccurate supervision :** 학습 데이터에 Noise가 있는 경우 (예: 개나 고양이의 라벨이 잘못 달려있는 경우)

약지도학습이란 위 3가지 경우를 포괄적으로 일컫도록 사용되곤 한다.

**inexact supervision**

일반적으로 Image Classification용 학습데이터보다는 bounding box 정보까지 정확하게 포함해야 하는 Object Detection이나 Semantic Segmentation을 위한 학습데이터가 훨씬 제작비용이 많이 든다.


Grad-CAM 논문에서 언급한 약지도학습을 통한 Object Detection과 Semantic Segmentation의 예시

![image](https://user-images.githubusercontent.com/70703320/115198321-e191e500-a12c-11eb-8547-dd870c67605c.png)

[출처 : https://arxiv.org/pdf/1610.02391.pdf]


위 사진은 Grad-CAM을 통한 Counterfactual Explanation 예시이다. 

**아이디어**

- Grad-CAM을 통해 개와 고양이의 특징이 두드러지게 하는 영역의 가중치 점수를 계산할 수 있었다 
- 여기서 해당 가중치 점수를 제거해 주면 Image classification 모델에서 해당 클래스에 대한 prediction이 바뀌게 될수도 있다. 
- 이와같이 제거했을 때 prediction이 바뀌도록 하는 가중치 영역을 모으면 한번도 bounding box 라벨을 보지 않고서도 object detection을 해낼 수 있다는 것이다.

![image](https://user-images.githubusercontent.com/70703320/115198364-ece51080-a12c-11eb-8568-a48291c0e8bf.png)

[출처 : https://arxiv.org/pdf/1610.02391.pdf]


또한 Grad-CAM을 이용해서, 위와 비슷한 논리로 Semantic Segmentation도 수행 가능함을 보여준다.



### 2) Adversarial Complementary Learning
---

![image](https://user-images.githubusercontent.com/70703320/115198421-fec6b380-a12c-11eb-812d-f5ce224b2eed.png)

http://openaccess.thecvf.com/content_cvpr_2018/papers/Zhang_Adversarial_Complementary_Learning_CVPR_2018_paper.pdf

ACoL은 모델의 학습에는 끝단이 두 브랜치(branch)로 나뉘는데 이는  CAM을 만들기 위해 활용했던 특성 맵을 두 가지로 분리한 것이다.

![image](https://user-images.githubusercontent.com/70703320/115198460-09814880-a12d-11eb-87d9-f951d9c0dc09.png)

http://openaccess.thecvf.com/content_cvpr_2018/papers/Zhang_Adversarial_Complementary_Learning_CVPR_2018_paper.pdf

**CAM, Grad-CAM**

Weaky-supervised 방식의 Object Detection, Segmentation 의 가능성이 존재하긴 한다.

**문제점**

- CAM을 통해 본 활성화 맵을 보면 가장자리보다는 **특징이 주로 나타나는 위치에 중점적으로 활성화**가 되는 모습을 볼 수 있다. 
- 하지만 물체 검출(Object detection)은 새의 부리나 동물의 눈, 시계의 숫자와 같이 부분적 특성이 아닌 물체의 전체적인 형태와 윤곽을 정확하게 구분해 내는 것이 중요하다. 



**ACol의 등장**

CAM 모델이 특정 부위에 집중해 학습하는 것을 막기 위해서 ACoL은 **브랜치를 두 가지로 두어 너무 높은 점수를 지워줌**으로서 주변의 특성 또한 반영하도록 했습니다. 


**주황색**
- 위쪽 주황색 브랜치를 거친다.
- 특성 맵은 GAP을 거쳐 CAM에서 보았던 소프트맥스 레이어인 `Classifier A`를 거쳐 이 브랜치는 loss로 학습된다. 

**ACoL의 역할**
- ACoL은 여기서 얻어진 활성화 맵을 적대적인 방법으로 사용하여 일정 값 이상 활성화된 활성화 맵을 지우도록 한다.

`Classifier A`:: 쉽게 전체적인 이미지를 보고 클래스를 판별할 수 있다. <br/>
`Classifier B`:: A의 CAM에서 크게 활성화된 영역을 지운 활성화 맵에서 분류

`Classifier A`와 `Classifier B`를 학습시킴으로써 더 넓은 영역을 판별의 근거로 삼을 수 있다. 이 과정을 통해서 모델은 쉽게 맞출 수 있는 샘플을 어렵게 다시 한 번 학습을 하는 Adversarial Complementary Learning 과정을 거치게 된다.

결과적으로 기존 CAM이 활성화 되는 부분과 달리 **ACoL은 물체의 전반적인 영역으로 CAM이 활성화되는 효과**를 확인할 수 있다.


### 3) 1x1 Conv
---

![image](https://user-images.githubusercontent.com/70703320/115198701-536a2e80-a12d-11eb-9717-213ec4a5bc50.png)
http://openaccess.thecvf.com/content_cvpr_2018/papers/Zhang_Adversarial_Complementary_Learning_CVPR_2018_paper.pdf

CAM에서는 CAM을 얻기 위해서 대상이 되는 네트워크에 feed forward를 하고 활성화 맵과 가중치 계산을 다시 해주어야 한다. 이 과정은 관찰하고자 하는 분류 모델의 feed forward와 별개의 작업이므로, 물체 검출을 위한 모델로 사용하기 위해서는 모델의 feed forward 외 별도의 연산을 해주어야 하는 단점이 있다.

ACoL 논문은 이를 개선하기 위해서 **커널 사이즈는 1x1, 출력 채널의 개수는 분류하고자 하는 클래스 개수**를 가진 컨볼루션 레이어를 특성 맵에 사용하고 여기에 GAP를 적용하여 Network in Network에서 본 구조와 유사한 방식을 사용하고 있다. 여기서 컨볼루션 레이어의 출력값은 곧바로 활성화 맵이 된다.